In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 1. 데이터 생성

In [0]:
# 웹 크롤링을 위한 패키지
import requests
import pandas_datareader as pdr
from pandas_datareader._utils import RemoteDataError
from io import BytesIO

# 분석 패키지
import pandas as pd
import numpy as np
import time
import os
# 다운로드 패키지
from google.colab import files

## 1.1 2015년 1월 6일을 기준으로 시가 총액 상위 230개의 기업 불러오기

In [0]:
gen_req_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx' # 거래소를 통해 웹크롤링

headers = {
'Referer': 'http://marketdata.krx.co.kr/mdi',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
}

query_str_parms = {
'name': 'fileDown',
'filetype': 'xls',
'url': 'MKD/04/0404/04040200/mkd04040200_01',
'market_gubun': 'STK',                    #시장 구분 - 코스피(STK)
'lmt_tp': '1',
'sect_tp_cd': 'ALL',
'schdate': '20150106',                    # 2015년 1월 6일을 기준 시점으로 선택
'pagePath': '/contents/MKD/04/0404/04040200/MKD04040200.jsp'
}

r = requests.get(gen_req_url, query_str_parms, headers=headers)

gen_req_url = 'http://file.krx.co.kr/download.jspx'

form_data = {
'code': r.content
}
r = requests.post(gen_req_url, form_data, headers=headers)

df = pd.read_excel(BytesIO(r.content))

In [0]:
df_200 = df.loc[:230] # 
type(df_200)

pandas.core.frame.DataFrame

In [0]:
df_200

,종목코드,종목명,현재가,대비,등락률,거래량,거래대금,시가,고가,저가,시가총액,시가총액비중(%),상장주식수,외국인 보유주식수,외국인 지분율(%)
0,005930,삼성전자,"1,295,000","-38,000",-2.8,"304,710","396,554,491,100","1,315,000","1,317,000","1,288,000","190,752,641,415,000",16.29,"147,299,337","76,180,644",51.72
1,005380,현대차,"164,500","-3,500",-2.1,"635,059","104,856,982,500","166,500","167,000","164,000","36,235,480,795,500",3.09,"220,276,479","96,147,766",43.65
2,000660,SK하이닉스,"47,500",-150,-0.3,"1,661,196","78,445,189,000","47,000","47,600","46,900","34,580,112,337,500",2.95,"728,002,365","363,692,827",49.96
3,015760,한국전력,"41,850",-150,-0.4,"1,290,400","53,942,971,468","41,500","42,400","41,400","26,866,196,622,450",2.29,"641,964,077","188,703,221",29.39
4,035420,NAVER,"743,000","-6,000",-0.8,"92,315","68,713,561,000","741,000","754,000","735,000","24,491,270,497,000",2.09,"32,962,679","17,464,824",52.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,001230,동국제강,"5,870",-170,-2.8,"264,704","1,566,646,770","6,010","6,070","5,850","521,398,582,300",0.04,"88,824,290","17,380,605",19.57
227,003090,대웅,"44,250","1,050",2.4,"1,333","58,756,200","43,850","44,350","42,500","514,556,523,000",0.04,"11,628,396","273,734",2.35
228,072710,농심홀딩스,"109,000","2,000",1.9,885,"96,138,500","108,500","109,500","107,500","505,519,110,000",0.04,"4,637,790","258,486",5.57
229,011160,두산건설,"9,050",-560,-5.8,"13,559","124,959,230","9,530","9,530","9,050","499,426,340,550",0.04,"55,185,231","461,048",0.84


In [0]:
df_code = df_200['종목코드']
df_name = df_200['종목명']
df_cap = df_200['시가총액']

code = df_code.to_list()
name = df_name.to_list()

## 1.2. (1.1)에서 정렬된 순서에 따라 주가 데이터 불러오기

In [0]:
m_week_11 = pd.date_range(start='20110101',end='20111231',freq='W-MON').strftime('%Y%m%d').to_list()
m_week_12 = pd.date_range(start='20120101',end='20121231',freq='W-MON').strftime('%Y%m%d').to_list()
m_week_13 = pd.date_range(start='20130101',end='20131231',freq='W-MON').strftime('%Y%m%d').to_list()
m_week_14 = pd.date_range(start='20140101',end='20141231',freq='W-MON').strftime('%Y%m%d').to_list()
m_week_15 = pd.date_range(start='20150101',end='20151231',freq='W-MON').strftime('%Y%m%d').to_list()
m_week_16 = pd.date_range(start='20160101',end='20161231',freq='W-MON').strftime('%Y%m%d').to_list()
m_week_17 = pd.date_range(start='20170101',end='20171231',freq='W-MON').strftime('%Y%m%d').to_list()
m_week_18 = pd.date_range(start='20180101',end='20181231',freq='W-MON').strftime('%Y%m%d').to_list()
m_week_19 = pd.date_range(start='20190101',end='20191231',freq='W-MON').strftime('%Y%m%d').to_list()

# 각 11~19년도 매주 월요일에 해당하는 날짜를 리스트 형태로 반환
# 11,13,14, 15, 16, 17, 19의 week는 52개
# 12,18 week는 53개

m_week = pd.DataFrame(pd.date_range(start='20110101',end='20191231',freq='W-MON'),columns=['Date'])
# 2011~2019 매주 월요일에 해당하는 날짜 리스트

t_week = pd.DataFrame(pd.date_range(start='20110101',end='20200103',freq='W-THU'),columns=['Date'])
# 2011~2019 매주 목요일에 해당하는 날짜 리스트
# 월요일이 휴일인 경우를 대비하여 결측치 대체를 위해 목요일 값을 사용함




In [0]:
m_week.shape

(470, 1)

In [0]:
week_num = []

for i in [11,12]:
    for j in range(1,53):
        num = '{}Y_week{}'.format(i,j)
        week_num.append(num)

week_num.append('12Y_week53')


for i in [13,14,15,16,17,18]:
    for j in range(1,53):
        num = '{}Y_week{}'.format(i,j)
        week_num.append(num)


week_num.append('18Y_week53')


for i in range(1,53):
    num = '19Y_week{}'.format(i)
    week_num.append(num)


len(week_num) == len(m_week)

# 470주 각각의 매주 월요일 날짜에 대해 14년 1주, 14년 2주, ... , 19년 52주까지 같은 길이의 리스트를 생성

True

In [0]:
def getdata(code,start_date,end_date):
    chart = pdr.get_data_yahoo(code,start_date,end_date) # pandas datareader를 이용한 크롤링, 가격과 거래량 정보만 획득
    chart.drop('Open',axis=1,inplace=True)
    chart.drop('High',axis=1,inplace=True)
    chart.drop('Low',axis=1,inplace=True)
    chart.drop('Close',axis=1,inplace=True)
    return(chart)

In [0]:
# 리스트 형태로 2011년부터 2019년까지, code정보에 해당하는 주가 정보를 리스트 형태로 저장

stock = []
stock_list=[]
start_date = '2011-01-01'
end_date = '2019-12-31'
for i,j in zip(code,name):
    try:
        a = getdata('{}.KS'.format(i),start_date,end_date)
        stock.append(a)
        stock_list.append(j)                           
    except KeyError:
        pass
    except RemoteDataError:
        pass

In [0]:
stock[0]

,Volume,Adj Close
Date,,
2011-01-03,13241050.0,13.779172
2011-01-04,13713100.0,13.779172
2011-01-05,16787500.0,13.549036
2011-01-06,19360400.0,13.376439
2011-01-07,23139800.0,13.246985
...,...,...
2019-12-23,9839252.0,55500.000000
2019-12-24,11868463.0,55000.000000
2019-12-26,9645034.0,55400.000000


In [0]:
len(stock)

220

In [0]:
len(stock) == len(stock_list)

True

In [0]:
# code : 종목코드 리스트
# name : 종목이름 리스트

# m_week : week 날짜의 문자열 데이터 프레임
# t_week : 결측치 대체를 위한 인덱싱 용도의 데이터 프레임
# week_num : {}Y_week{} 형식의 날짜 리스트

# stock : [2011년부터 2019년까지] [Daily 주가 데이터]
# stock_list : Crawling 데이터 중 기간 내 상장된 회사의 목록, stock value의 label


## 1.3. Daily 데이터를 Weekly로 변환하기

In [0]:
Key = pd.DataFrame(stock[0].index.strftime('%Y%m%d').astype(int),columns=['Key'])

k = []

for i,j in zip(stock_list,np.array(range(len(stock)))):
        n = pd.DataFrame(stock[j])
        m = n.join(Key)
        m['Key'] = Key
        m.drop('Volume',axis=1,inplace=True)
        m.rename(columns = {'Adj Close':'{}'.format(i)},inplace=True)
        k.append(m)

df_price = pd.concat(k,axis=1)
df_price.drop('Key',axis=1,inplace=True)
df_price['Date']=np.array(df_price.index)

df_price.shape

In [0]:
m_week.shape

(470, 1)

In [0]:
type(week_num)

list

In [0]:
week_price = pd.merge(m_week,df_price,how='left')
week_price.index = week_num
week_price['Week'] = week_num

In [0]:
week_price

,Date,삼성전자,현대차,SK하이닉스,한국전력,NAVER,POSCO,삼성생명,삼성전자우,삼성에스디에스,현대모비스,SK텔레콤,기아차,신한지주,제일모직,삼성화재,KB금융,아모레퍼시픽,LG디스플레이,SK C&C,현대글로비스,LG화학,KT&G,LG,LG전자,LG생활건강,하나금융지주,KT,롯데쇼핑,아모레G,현대중공업,삼성SDI,기업은행,고려아연,현대제철,SK이노베이션,한화생명,강원랜드,코웨이,한국타이어,...,빙그레,한진,신도리코,동아에스티,풍산,현대상사,한국단자,SBS미디어홀딩스,대상홀딩스,도레이케미칼,다우기술,지역난방공사,NICE,종근당,현대리바트,고려제강,락앤락,에스엘,서흥,대교,S&T모티브,LG생활건강우,동원시스템즈,팬오션,광동제약,대한해운,부광약품,삼천리,한국쉘석유,SBS,삼성화재우,세방전지,동아쏘시오홀딩스,신영증권,동국제강,대웅,농심홀딩스,두산건설,광주은행,Week
11Y_week1,2011-01-03,13.779172,151181.28125,23364.677734,26055.728516,51203.554688,382628.40625,89217.890625,0.023668,NaN,273873.187500,119694.320312,45051.210938,45645.898438,NaN,183997.250000,52807.847656,88669.742188,38141.984375,80369.734375,138631.953125,344600.71875,50461.187500,77877.390625,112318.007812,3.760554e+05,36156.273438,36811.546875,444921.15625,17328.509766,291021.90625,166478.218750,15060.157227,254742.218750,115202.515625,155055.562500,6804.369629,23502.968750,31911.369141,NaN,...,48920.242188,30753.775391,55100.0,NaN,41714.382812,18456.019531,18362.763672,2526.526855,2865.511475,NaN,8044.161133,78000.0,4219.982910,NaN,7059.741211,22210.513672,32577.306641,16142.135742,7505.878418,4835.841309,26786.833984,81917.695312,15846.634766,144218.390625,3240.954102,1980000.0,3333.544678,94094.468750,149324.312500,25139.630859,61156.863281,31789.736328,88486.671875,25702.587891,27524.031250,1928.700439,49407.046875,59055.687500,NaN,11Y_week1
11Y_week2,2011-01-10,13.189456,169118.06250,23364.677734,25232.005859,49700.910156,384596.68750,94365.078125,0.022841,NaN,284102.687500,118306.523438,51229.652344,43176.257812,NaN,186461.484375,49646.214844,94517.843750,36274.566406,81741.226562,151026.187500,359161.31250,49232.304688,78233.000000,107638.000000,3.911732e+05,35951.769531,35719.339844,433657.31250,17129.333984,304401.59375,162737.156250,14432.652344,256073.609375,118874.296875,148677.953125,7021.976074,24661.566406,30520.466797,NaN,...,51406.207031,30753.775391,55400.0,NaN,42406.734375,19652.591797,19431.986328,2665.294678,2817.018066,NaN,8414.006836,78600.0,4246.819336,NaN,6905.850098,22511.902344,32986.218750,18702.925781,7376.147461,4993.104004,28014.378906,82791.945312,15603.587891,142345.421875,3231.600586,2002500.0,3345.844971,93672.523438,153766.453125,24447.710938,61851.832031,32262.091797,85732.203125,26006.556641,27878.357422,1961.394775,49237.554688,58693.945312,NaN,11Y_week2
11Y_week3,2011-01-17,13.649718,169118.06250,24842.285156,24278.216797,48776.218750,369637.96875,93936.132812,0.022738,NaN,286892.593750,115877.929688,49685.042969,45305.257812,NaN,180711.578125,52380.601562,85824.726562,34874.003906,87318.648438,148271.921875,366220.96875,48771.464844,76010.468750,108105.421875,3.741657e+05,39264.730469,34546.226562,416761.56250,15854.589844,333838.62500,156657.875000,14040.458008,263618.218750,117038.398438,145489.171875,6879.695312,23792.619141,29248.777344,NaN,...,48476.320312,31748.750000,53200.0,NaN,43055.816406,20123.947266,18316.271484,2593.518066,2768.524902,NaN,8321.543945,74900.0,4173.020020,NaN,6925.086914,21939.261719,32259.285156,18245.644531,7385.414062,4820.115234,27795.175781,80868.593750,15166.104492,146715.671875,2974.381836,2073750.0,3395.045410,90718.882812,164017.546875,24447.710938,61017.867188,32451.035156,79879.007812,26378.082031,27090.841797,1945.047241,48729.078125,64843.632812,NaN,11Y_week3
11Y_week4,2011-01-24,13.966155,160149.68750,26181.367188,24625.048828,48545.097656,373180.81250,91791.484375,0.022566,NaN,267828.437500,116571.843750,48312.050781,42154.328125,NaN,181943.703125,48706.265625,84402.226562,36134.511719,88050.117188,145517.640625,351219.25000,46697.718750,76810.585938,115125.414062,3.751106e+05,38037.710938,35031.656250,411129.65625,15336.727539,318785.75000,149643.359375,13138.417969,280926.531250,119333.257812,151866.765625,6729.044434,21765.076172,27698.908203,NaN,...,48387.531250,32201.015625,52600.0,NaN,40372.937500,21102.937500,18130.324219,2373.404053,2702.397949,NaN,8192.097656,74700.0,4253.528320,NaN,7204.013672,21878.9

In [0]:
# null 값 찾기

week_price.loc[week_price.isnull()['삼성전자'],:].index

Index(['11Y_week23', '11Y_week33', '11Y_week37', '11Y_week40', '12Y_week4',
       '12Y_week22', '12Y_week40', '12Y_week53', '13Y_week6', '14Y_week18',
       '14Y_week36', '15Y_week21', '15Y_week39', '16Y_week6', '16Y_week23',
       '16Y_week33', '16Y_week40', '17Y_week5', '17Y_week18', '17Y_week40',
       '17Y_week41', '17Y_week52', '18Y_week1', '18Y_week19', '18Y_week39',
       '18Y_week53', '19Y_week5', '19Y_week18'],
      dtype='object')

In [0]:
week_price.loc[week_price.isnull()['삼성전자'],:]

,Date,삼성전자,현대차,SK하이닉스,한국전력,NAVER,POSCO,삼성생명,삼성전자우,삼성에스디에스,현대모비스,SK텔레콤,기아차,신한지주,제일모직,삼성화재,KB금융,아모레퍼시픽,LG디스플레이,SK C&C,현대글로비스,LG화학,KT&G,LG,LG전자,LG생활건강,하나금융지주,KT,롯데쇼핑,아모레G,현대중공업,삼성SDI,기업은행,고려아연,현대제철,SK이노베이션,한화생명,강원랜드,코웨이,한국타이어,...,빙그레,한진,신도리코,동아에스티,풍산,현대상사,한국단자,SBS미디어홀딩스,대상홀딩스,도레이케미칼,다우기술,지역난방공사,NICE,종근당,현대리바트,고려제강,락앤락,에스엘,서흥,대교,S&T모티브,LG생활건강우,동원시스템즈,팬오션,광동제약,대한해운,부광약품,삼천리,한국쉘석유,SBS,삼성화재우,세방전지,동아쏘시오홀딩스,신영증권,동국제강,대웅,농심홀딩스,두산건설,광주은행,Week
11Y_week23,2011-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11Y_week23
11Y_week33,2011-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11Y_week33
11Y_week37,2011-09-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11Y_week37
11Y_week40,2011-10-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11Y_week40
12Y_week4,2012-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12Y_week4
12Y_week22,2012-05-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12Y_week22
12Y_week40,2012-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12Y_week40
12Y_week53,2012-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12Y_week53
13Y_week6,2013-02-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13Y_week6
14Y_week18,2014-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14Y_week18


In [0]:
nan_list = ['11Y_week23', '11Y_week33', '11Y_week37', '11Y_week40', '12Y_week4',
       '12Y_week22', '12Y_week40', '12Y_week53', '13Y_week6', '14Y_week18',
       '14Y_week36', '15Y_week21', '15Y_week39', '16Y_week6', '16Y_week23',
       '16Y_week33', '16Y_week40', '17Y_week5', '17Y_week18', '17Y_week40',
       '17Y_week41', '17Y_week52', '18Y_week1', '18Y_week19', '18Y_week39',
       '18Y_week53', '19Y_week5', '19Y_week18']

In [0]:
t_week

# 월요일이 휴일인 날을 대신하여 목요일 날짜 기준으로 새로운 주별 pd.Series 생성하기

,Date
0,2011-01-06
1,2011-01-13
2,2011-01-20
3,2011-01-27
4,2011-02-03
...,...
465,2019-12-05
466,2019-12-12
467,2019-12-19
468,2019-12-26


In [0]:
add_price = pd.merge(t_week,df_price,how='left')
add_price.index = week_num
add_price['Week'] = week_num

for i in nan_list:
  week_price.loc[i] = add_price.loc[i]

  
# 목요일 데이터로 결측치 대체하기

In [0]:
week_price.loc[week_price.isnull()['삼성전자'],:].index

Index(['17Y_week40'], dtype='object')

In [0]:
week_price.drop('17Y_week40',inplace=True)

# 17년도 40째 주는 한 주가 추석 전체 휴일로 영업하는 날이 없어 삭제

In [0]:
week_num2 = week_num.copy()
week_num2.remove('17Y_week40')

In [0]:
tmp = week_price.drop('Date',axis=1)
tmp_price = tmp.drop('Week',axis=1)

week_return = np.log(tmp_price / tmp_price.shift(1))
week_return['Date'] = week_price['Date']
week_return['Week'] = week_num

week_return

# 로그 수익률로 수익률 계산하기

,삼성전자,현대차,SK하이닉스,한국전력,NAVER,POSCO,삼성생명,삼성전자우,삼성에스디에스,현대모비스,SK텔레콤,기아차,신한지주,제일모직,삼성화재,KB금융,아모레퍼시픽,LG디스플레이,SK C&C,현대글로비스,LG화학,KT&G,LG,LG전자,LG생활건강,하나금융지주,KT,롯데쇼핑,아모레G,현대중공업,삼성SDI,기업은행,고려아연,현대제철,SK이노베이션,한화생명,강원랜드,코웨이,한국타이어,오리온,...,한진,신도리코,동아에스티,풍산,현대상사,한국단자,SBS미디어홀딩스,대상홀딩스,도레이케미칼,다우기술,지역난방공사,NICE,종근당,현대리바트,고려제강,락앤락,에스엘,서흥,대교,S&T모티브,LG생활건강우,동원시스템즈,팬오션,광동제약,대한해운,부광약품,삼천리,한국쉘석유,SBS,삼성화재우,세방전지,동아쏘시오홀딩스,신영증권,동국제강,대웅,농심홀딩스,두산건설,광주은행,Date,Week
11Y_week1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-01-03,11Y_week1
11Y_week2,-0.043740,0.112117,0.000000,-0.032124,-0.029786,0.005131,0.056089,-0.035559,NaN,0.036671,-0.011662,0.128519,-0.055623,NaN,0.013304,-0.061738,0.063870,-0.050199,0.016921,0.085631,0.041385,-0.024654,0.004556,-0.042560,0.039414,-0.005672,-0.030119,-0.025642,-0.011561,0.044949,-0.022728,-0.042559,0.005213,0.031375,-0.042001,0.031480,0.048119,-0.044565,NaN,0.048847,...,0.000000,0.005430,NaN,0.016461,0.062819,0.056596,0.053469,-0.017068,NaN,0.044951,0.007663,0.006339,NaN,-0.022040,0.013478,0.012474,0.147247,-0.017435,0.032003,0.044807,0.010616,-0.015456,-0.013072,-0.002890,0.011300,0.003683,-0.004494,0.029314,-0.027909,0.011300,0.014749,-0.031623,0.011757,0.012791,0.016809,-0.003436,-0.006144,NaN,2011-01-10,11Y_week2
11Y_week3,0.034301,0.000000,0.061322,-0.038534,-0.018780,-0.039671,-0.004556,-0.004535,NaN,0.009772,-0.020742,-0.030615,0.048132,NaN,-0.031322,0.053614,-0.096481,-0.039375,0.066006,-0.018405,0.019465,-0.009405,-0.028820,0.004333,-0.044452,0.088148,-0.033394,-0.039740,-0.077333,0.092310,-0.038072,-0.027550,0.029037,-0.015565,-0.021681,-0.020470,-0.035871,-0.042560,NaN,0.039725,...,0.031841,-0.040521,NaN,0.015190,0.023701,-0.059131,-0.027299,-0.017364,NaN,-0.011050,-0.048218,-0.017530,NaN,0.002782,-0.025766,-0.022284,-0.024754,0.001256,-0.035260,-0.007855,-0.023505,-0.028438,0.030240,-0.082941,0.034962,0.014598,-0.032039,0.064539,0.000000,-0.013575,0.005839,-0.070715,0.014185,-0.028655,-0.008370,-0.010381,0.099642,NaN,2011-01-17,11Y_week3
11Y_week4,0.022918,-0.054488,0.052501,0.014185,-0.004750,0.009539,-0.023096,-0.007604,NaN,-0.068761,0.005970,-0.028023,-0.072086,NaN,0.006795,-0.072729,-0.016713,0.035507,0.008342,-0.018751,-0.041826,-0.043450,0.010471,0.062915,0.002522,-0.031749,0.013954,-0.013606,-0.033209,-0.046138,-0.045809,-0.066402,0.063591,0.019418,0.042902,-0.022141,-0.089069,-0.054445,NaN,-0.042111,...,0.014145,-0.011342,NaN,-0.064338,0.047502,-0.010204,-0.088690,-0.024175,NaN,-0.015678,-0.002674,0.019109,NaN,0.039488,-0.002751,-0.027125,0.090972,0.003757,-0.014791,-0.032054,0.021391,0.012739,-0.034635,-0.011067,-0.082910,-0.040670,-0.004662,-0.023183,-0.044366,0.005679,0.032931,-0.004319,-0.031212,-0.017595,-0.031298,-0.015776,-0.050045,NaN,2011-01-24,11Y_week4
11Y_week5,0.010246,-0.046393,0.044835,0.000000,-0.048789,-0.043110,-0.038100,0.015152,NaN,-0.108065,-0.021053,-0.028831,0.002018,NaN,0.044157,0.008734,0.017634,-0.014314,0.057505,-0.015899,0.053802,-0.028359,-0.013986,-0.050010,0.024877,-0.046213,-0.035256,0.033673,-0.023654,0.017680,-0.025318,0.011870,-0.063591,0.066940,0.070916,-0.030305,-0.013397,0.001433,NaN,0.001196,...,-0.065288,0.020697,NaN,-0.053948,-0.010364,-0.002568,-0.020367,-0.028124,NaN,0.052760,-0.021652,0.020297,NaN,-0.042269,0.002751,-0.005807,-0.150344,-0.051293,0.006601,0.075271,0.004224,-0.015949,-0.045053,-0.025766,NaN,-0.026771,-0.004684,-0.054778,0.055623,0.004520,0.013986,-0.017467,-0.020014,0.060277,0.000000,-0.030496,-0.111571,NaN,2011-01-31,11Y_week5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [0]:
week_return.isnull().sum()

삼성전자        3
현대차         3
SK하이닉스      3
한국전력        3
NAVER       3
         ... 
농심홀딩스       3
두산건설        3
광주은행      443
Date        0
Week        0
Length: 222, dtype: int64

In [0]:
week_return.loc[week_return.isnull()['삼성전자'],:].index

Index(['11Y_week1', '17Y_week40', '17Y_week41'], dtype='object')

## 1.4. 연도별 시가 총액 구하기

In [0]:
gen_req_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'

headers = {
'Referer': 'http://marketdata.krx.co.kr/mdi',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
}

query_str_parms = {
'name': 'fileDown',
'filetype': 'xls',
'url': 'MKD/04/0404/04040200/mkd04040200_01',
'market_gubun': 'STK',
'lmt_tp': '1',
'sect_tp_cd': 'ALL',
'schdate': '20140106',
'pagePath': '/contents/MKD/04/0404/04040200/MKD04040200.jsp'
}

r = requests.get(gen_req_url, query_str_parms, headers=headers)

gen_req_url = 'http://file.krx.co.kr/download.jspx'

form_data = {
'code': r.content
}
r = requests.post(gen_req_url, form_data, headers=headers)

df_14 = pd.read_excel(BytesIO(r.content))

In [0]:
gen_req_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'

headers = {
'Referer': 'http://marketdata.krx.co.kr/mdi',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
}

query_str_parms = {
'name': 'fileDown',
'filetype': 'xls',
'url': 'MKD/04/0404/04040200/mkd04040200_01',
'market_gubun': 'STK',
'lmt_tp': '1',
'sect_tp_cd': 'ALL',
'schdate': '20150102',
'pagePath': '/contents/MKD/04/0404/04040200/MKD04040200.jsp'
}

r = requests.get(gen_req_url, query_str_parms, headers=headers)

gen_req_url = 'http://file.krx.co.kr/download.jspx'

form_data = {
'code': r.content
}
r = requests.post(gen_req_url, form_data, headers=headers)

df_15 = pd.read_excel(BytesIO(r.content))

In [0]:
gen_req_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'

headers = {
'Referer': 'http://marketdata.krx.co.kr/mdi',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
}

query_str_parms = {
'name': 'fileDown',
'filetype': 'xls',
'url': 'MKD/04/0404/04040200/mkd04040200_01',
'market_gubun': 'STK',
'lmt_tp': '1',
'sect_tp_cd': 'ALL',
'schdate': '20160106',
'pagePath': '/contents/MKD/04/0404/04040200/MKD04040200.jsp'
}

r = requests.get(gen_req_url, query_str_parms, headers=headers)

gen_req_url = 'http://file.krx.co.kr/download.jspx'

form_data = {
'code': r.content
}
r = requests.post(gen_req_url, form_data, headers=headers)

df_16 = pd.read_excel(BytesIO(r.content))

In [0]:
gen_req_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'

headers = {
'Referer': 'http://marketdata.krx.co.kr/mdi',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
}

query_str_parms = {
'name': 'fileDown',
'filetype': 'xls',
'url': 'MKD/04/0404/04040200/mkd04040200_01',
'market_gubun': 'STK',
'lmt_tp': '1',
'sect_tp_cd': 'ALL',
'schdate': '20170106',
'pagePath': '/contents/MKD/04/0404/04040200/MKD04040200.jsp'
}

r = requests.get(gen_req_url, query_str_parms, headers=headers)

gen_req_url = 'http://file.krx.co.kr/download.jspx'

form_data = {
'code': r.content
}
r = requests.post(gen_req_url, form_data, headers=headers)

df_17 = pd.read_excel(BytesIO(r.content))

In [0]:
gen_req_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'

headers = {
'Referer': 'http://marketdata.krx.co.kr/mdi',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
}

query_str_parms = {
'name': 'fileDown',
'filetype': 'xls',
'url': 'MKD/04/0404/04040200/mkd04040200_01',
'market_gubun': 'STK',
'lmt_tp': '1',
'sect_tp_cd': 'ALL',
'schdate': '20180102',
'pagePath': '/contents/MKD/04/0404/04040200/MKD04040200.jsp'
}

r = requests.get(gen_req_url, query_str_parms, headers=headers)

gen_req_url = 'http://file.krx.co.kr/download.jspx'

form_data = {
'code': r.content
}
r = requests.post(gen_req_url, form_data, headers=headers)

df_18 = pd.read_excel(BytesIO(r.content))

In [0]:
gen_req_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'

headers = {
'Referer': 'http://marketdata.krx.co.kr/mdi',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
}

query_str_parms = {
'name': 'fileDown',
'filetype': 'xls',
'url': 'MKD/04/0404/04040200/mkd04040200_01',
'market_gubun': 'STK',
'lmt_tp': '1',
'sect_tp_cd': 'ALL',
'schdate': '20190102',
'pagePath': '/contents/MKD/04/0404/04040200/MKD04040200.jsp'
}

r = requests.get(gen_req_url, query_str_parms, headers=headers)

gen_req_url = 'http://file.krx.co.kr/download.jspx'

form_data = {
'code': r.content
}
r = requests.post(gen_req_url, form_data, headers=headers)

df_19 = pd.read_excel(BytesIO(r.content))

In [0]:
# 각 연도별 연초를 기준으로 유통 주식수를 고정하고, 종목명에 맞춰 데이터를 생성

tmp_df = pd.DataFrame(week_price.columns,columns=['종목명'])

df_14_tmp = pd.merge(tmp_df,df_14,how='left')
df_15_tmp = pd.merge(tmp_df,df_15,how='left')
df_16_tmp = pd.merge(tmp_df,df_16,how='left')
df_17_tmp = pd.merge(tmp_df,df_17,how='left')
df_18_tmp = pd.merge(tmp_df,df_18,how='left')
df_19_tmp = pd.merge(tmp_df,df_19,how='left')

In [0]:
week_price_tmp = week_price.iloc[157:].copy()
week_price_tmp.drop('Date',axis=1,inplace=True)
week_price_tmp.drop('Week',axis=1,inplace=True)
week_price_tmp.drop('17Y_week40',inplace=True)

In [0]:
n_stocks = pd.DataFrame(index = week_price_tmp.index,columns =week_price_tmp.columns)

for i in np.array(range(0,52)):
  for j in np.array(range(0,220)):
    n_stocks.iloc[i,j] = df_14_tmp.iloc[j+1,12]
    n_stocks.iloc[i+52,j] = df_15_tmp.iloc[j+1,12]
    n_stocks.iloc[i+104,j] = df_16_tmp.iloc[j+1,12]
    n_stocks.iloc[i+156,j] = df_17_tmp.iloc[j+1,12]
    n_stocks.iloc[i+208,j] = df_18_tmp.iloc[j+1,12]
    n_stocks.iloc[i+260,j] = df_16_tmp.iloc[j+1,12]

In [0]:
def remove_comma(x):
  return x.replace(',','')

In [0]:
for i in n_stocks.columns:
  try:
    n_stocks['{}'.format(i)] = n_stocks['{}'.format(i)].apply(remove_comma)
  except AttributeError:
    pass

for i in n_stocks.columns:
  try:
    n_stocks['{}'.format(i)] = n_stocks['{}'.format(i)].astype('float')
  except ValueError:
    pass

In [0]:
n_stocks

# 15 230개 기업의 14~19년 유통 주식수

,삼성전자,현대차,SK하이닉스,한국전력,NAVER,POSCO,삼성생명,삼성전자우,삼성에스디에스,현대모비스,SK텔레콤,기아차,신한지주,제일모직,삼성화재,KB금융,아모레퍼시픽,LG디스플레이,SK C&C,현대글로비스,LG화학,KT&G,LG,LG전자,LG생활건강,하나금융지주,KT,롯데쇼핑,아모레G,현대중공업,삼성SDI,기업은행,고려아연,현대제철,SK이노베이션,한화생명,강원랜드,코웨이,한국타이어,오리온,...,세아홀딩스,빙그레,한진,신도리코,동아에스티,풍산,현대상사,한국단자,SBS미디어홀딩스,대상홀딩스,도레이케미칼,다우기술,지역난방공사,NICE,종근당,현대리바트,고려제강,락앤락,에스엘,서흥,대교,S&T모티브,LG생활건강우,동원시스템즈,팬오션,광동제약,대한해운,부광약품,삼천리,한국쉘석유,SBS,삼성화재우,세방전지,동아쏘시오홀딩스,신영증권,동국제강,대웅,농심홀딩스,두산건설,광주은행
14Y_week1,147299337.0,220276479.0,710200891.0,641964077.0,32962679.0,87186835.0,200000000.0,22833427.0,NaN,97343863.0,80745711.0,405363347.0,474199587.0,"52,437,651",47374837.0,386351693.0,5845849.0,357815700.0,"50,000,000",37500000.0,66271100.0,137292497.0,172557131.0,163647814.0,15618197.0,289894062.0,261111808.0,31490892.0,7979098.0,76000000.0,45558341.0,550195005.0,18870000.0,85314475.0,92465564.0,868530000.0,213940500.0,77124796.0,123875069.0,5971135.0,...,4000000.0,9851241.0,11974656.0,10080029.0,7350916.0,28024278.0,22329802.0,10415000.0,139896190.0,36212538.0,NaN,44866617.0,11578744.0,37882300.0,9409050.0,NaN,15000000.0,55000000.0,33865090.0,NaN,84702850.0,14403386.0,2099697.0,24079554.0,NaN,52420851.0,23955512.0,29687321.0,4055025.0,1300000.0,18252582.0,3192000.0,14000000.0,4339639.0,9386237.0,61824290.0,11628396.0,4637790.0,551852310.0,NaN
14Y_week2,147299337.0,220276479.0,710200891.0,641964077.0,32962679.0,87186835.0,200000000.0,22833427.0,NaN,97343863.0,80745711.0,405363347.0,474199587.0,"52,437,651",47374837.0,386351693.0,5845849.0,357815700.0,"50,000,000",37500000.0,66271100.0,137292497.0,172557131.0,163647814.0,15618197.0,289894062.0,261111808.0,31490892.0,7979098.0,76000000.0,45558341.0,550195005.0,18870000.0,85314475.0,92465564.0,868530000.0,213940500.0,77124796.0,123875069.0,5971135.0,...,4000000.0,9851241.0,11974656.0,10080029.0,7350916.0,28024278.0,22329802.0,10415000.0,139896190.0,36212538.0,NaN,44866617.0,11578744.0,37882300.0,9409050.0,NaN,15000000.0,55000000.0,33865090.0,NaN,84702850.0,14403386.0,2099697.0,24079554.0,NaN,52420851.0,23955512.0,29687321.0,4055025.0,1300000.0,18252582.0,3192000.0,14000000.0,4339639.0,9386237.0,61824290.0,11628396.0,4637790.0,551852310.0,NaN
14Y_week3,147299337.0,220276479.0,710200891.0,641964077.0,32962679.0,87186835.0,200000000.0,22833427.0,NaN,97343863.0,80745711.0,405363347.0,474199587.0,"52,437,651",47374837.0,386351693.0,5845849.0,357815700.0,"50,000,000",37500000.0,66271100.0,137292497.0,172557131.0,163647814.0,15618197.0,289894062.0,261111808.0,31490892.0,7979098.0,76000000.0,45558341.0,550195005.0,18870000.0,85314475.0,92465564.0,868530000.0,213940500.0,77124796.0,123875069.0,5971135.0,...,4000000.0,9851241.0,11974656.0,10080029.0,7350916.0,28024278.0,22329802.0,10415000.0,139896190.0,36212538.0,NaN,44866617.0,11578744.0,37882300.0,9409050.0,NaN,15000000.0,55000000.0,33865090.0,NaN,84702850.0,14403386.0,2099697.0,24079554.0,NaN,52420851.0,23955512.0,29687321.0,4055025.0,1300000.0,18252582.0,3192000.0,14000000.0,4339639.0,9386237.0,61824290.0,11628396.0,4637790.0,551852310.0,NaN
14Y_week4,147299337.0,220276479.0,710200891.0,641964077.0,32962679.0,87186835.0,200000000.0,22833427.0,NaN,97343863.0,80745711.0,405363347.0,474199587.0,"52,437,651",47374837.0,386351693.0,5845849.0,357815700.0,"50,000,000",37500000.0,66271100.0,137292497.0,172557131.0,163647814.0,15618197.0,289894062.0,261111808.0,31490892.0,7979098.0,76000000.0,45558341.0,550195005.0,18870000.0,85314475.0,92465564.0,868530000.0,213940500.0,77124796.0,123875069.0,5971135.0,...,4000000.0,9851241.0,11974656.0,10080029.0,7350916.0,28024278.0,22329802.0,10415000.0,139896190.0,36212538.0,NaN,44866617.0,11578744.0,37882300.0,9409050.0,NaN,15000000.0,55000000.0,33865090.0,NaN,84702850.0,14403386.0,2099697.0,24079554.0,NaN,52420851.0,23955512.0,29687321.0,4055025.0,1300000.0,18252582.0,3192000.0,14000000.0,4339639.0,9386237.0,61824290.0,11628396.0,4637790.0,551852310.0,NaN
14Y_week5,147299337.0,220276479.0,710200891.0,641964077.0,32962679.0,87186835.0,200000000.0,22833427.0,NaN,97343863.

In [0]:
week_cap = pd.DataFrame(index=week_price_tmp.index,columns=week_price_tmp.columns)

for i in range(len(week_price_tmp.index)):
  for j in range(len(week_price_tmp.columns)):
    try:
      week_cap.iloc[i,j] = week_price_tmp.iloc[i,j] * asd.iloc[i,j]
    except TypeError:
      pass

In [0]:
week_cap
# 14~19년도 주가 * 유통 주식수(연초)를 통해 계산한 시가총액

,삼성전자,현대차,SK하이닉스,한국전력,NAVER,POSCO,삼성생명,삼성전자우,삼성에스디에스,현대모비스,SK텔레콤,기아차,신한지주,제일모직,삼성화재,KB금융,아모레퍼시픽,LG디스플레이,SK C&C,현대글로비스,LG화학,KT&G,LG,LG전자,LG생활건강,하나금융지주,KT,롯데쇼핑,아모레G,현대중공업,삼성SDI,기업은행,고려아연,현대제철,SK이노베이션,한화생명,강원랜드,코웨이,한국타이어,오리온,...,세아홀딩스,빙그레,한진,신도리코,동아에스티,풍산,현대상사,한국단자,SBS미디어홀딩스,대상홀딩스,도레이케미칼,다우기술,지역난방공사,NICE,종근당,현대리바트,고려제강,락앤락,에스엘,서흥,대교,S&T모티브,LG생활건강우,동원시스템즈,팬오션,광동제약,대한해운,부광약품,삼천리,한국쉘석유,SBS,삼성화재우,세방전지,동아쏘시오홀딩스,신영증권,동국제강,대웅,농심홀딩스,두산건설,광주은행
14Y_week1,1.02157e+10,4.41012e+13,2.48112e+13,1.96735e+13,4.46196e+12,2.33601e+13,1.85401e+13,7.73655e+06,NaN,2.57927e+13,1.51036e+13,1.8877e+13,1.93448e+13,NaN,1.01451e+13,1.39388e+13,5.43096e+11,8.43591e+12,NaN,7.875e+12,1.7285e+13,8.73129e+12,9.81344e+12,1.02966e+13,8.14264e+12,1.05144e+13,7.53063e+12,1.21184e+13,3.60144e+11,1.22558e+13,6.50251e+12,5.71415e+12,5.55977e+12,6.53974e+12,1.07754e+13,5.74993e+12,6.36964e+12,4.21408e+12,7.11475e+12,2.63694e+11,...,3.48876e+11,8.0068e+11,2.31865e+11,6.37058e+11,7.55535e+11,7.04178e+11,5.99654e+11,4.15522e+11,7.01687e+11,2.8211e+11,NaN,6.4328e+11,9.70299e+11,4.57963e+11,5.3968e+11,NaN,3.92075e+11,1.18538e+12,5.05388e+11,NaN,5.17099e+11,3.40137e+11,4.43548e+11,2.02027e+11,NaN,3.86899e+11,6.91117e+11,1.56448e+11,4.78639e+11,4.9073e+11,7.14422e+11,4.06574e+11,7.29123e+11,5.8792e+11,2.90328e+11,6.61177e+11,7.84425e+10,3.49806e+11,8.8062e+12,NaN
14Y_week2,1.01219e+10,4.40047e+13,2.51407e+13,1.97014e+13,4.27685e+12,2.29172e+13,1.83574e+13,7.52491e+06,NaN,2.57927e+13,1.49711e+13,1.8877e+13,1.89626e+13,NaN,1.02486e+13,1.38003e+13,5.44204e+11,8.61966e+12,NaN,7.83984e+12,1.64041e+13,8.93351e+12,9.78127e+12,1.03754e+13,8.09749e+12,1.01936e+13,7.63941e+12,1.20436e+13,3.61294e+11,1.14867e+13,6.63432e+12,5.8582e+12,5.551e+12,6.32467e+12,1.03052e+13,5.79014e+12,6.35042e+12,4.26039e+12,7.64355e+12,2.51668e+11,...,3.46656e+11,8.13491e+11,2.45571e+11,6.16898e+11,7.94923e+11,6.78802e+11,5.80561e+11,4.11512e+11,7.08526e+11,2.79708e+11,NaN,6.79612e+11,9.70299e+11,4.83712e+11,5.74044e+11,NaN,3.92617e+11,1.20077e+12,5.35872e+11,NaN,5.38614e+11,3.52189e+11,4.46492e+11,1.97579e+11,NaN,4.04139e+11,6.42008e+11,1.55331e+11,4.73073e+11,4.83105e+11,7.04684e+11,3.84562e+11,7.25072e+11,5.75277e+11,2.91084e+11,6.47795e+11,7.6474e+10,3.51542e+11,8.78044e+12,NaN
14Y_week3,1.0286e+10,4.45837e+13,2.42841e+13,1.94223e+13,4.57686e+12,2.29541e+13,1.84487e+13,7.35246e+06,NaN,2.64848e+13,1.44743e+13,1.81938e+13,1.82619e+13,NaN,1.02072e+13,1.35233e+13,5.64709e+11,8.78671e+12,NaN,8.12109e+12,1.63433e+13,8.87404e+12,9.70083e+12,1.08477e+13,8.41356e+12,9.89739e+12,7.43392e+12,1.14002e+13,3.60144e+11,1.15363e+13,6.81006e+12,5.97824e+12,5.94562e+12,6.29281e+12,1.01093e+13,5.74993e+12,6.48493e+12,4.37947e+12,7.53539e+12,2.6164e+11,...,3.45546e+11,8.39113e+11,2.80408e+11,6.23954e+11,7.94923e+11,6.94028e+11,5.73619e+11,4.04495e+11,7.00319e+11,2.79021e+11,NaN,6.56103e+11,8.79985e+11,5.14979e+11,5.6311e+11,NaN,3.93158e+11,1.16485e+12,5.45498e+11,NaN,5.3713e+11,3.52189e+11,4.23922e+11,2.01558e+11,NaN,3.90448e+11,6.28832e+11,1.54772e+11,4.87915e+11,4.26461e+11,7.44522e+11,3.80677e+11,7.26422e+11,5.37346e+11,2.90328e+11,6.39761e+11,7.86393e+10,3.51976e+11,8.78044e+12,NaN
14Y_week4,1.00984e+10,4.31361e+13,2.32297e+13,1.8641e+13,4.20663e+12,2.20685e+13,1.82113e+13,7.43868e+06,NaN,2.71308e+13,1.36131e+13,1.92365e+13,1.88564e+13,NaN,1.00001e+13,1.27614e+13,5.6748e+11,8.7366e+12,NaN,7.62891e+12,1.5918e+13,8.9573e+12,9.55604e+12,1.08477e+13,6.8633e+12,9.92207e+12,7.21635e+12,1.09963e+13,3.49417e+11,1.11394e+13,6.28283e+12,5.8582e+12,5.96316e+12,6.02995e+12,1.00701e+13,5.60518e+12,6.45611e+12,4.23393e+12,7.39117e+12,2.51961e+11,...,3.41476e+11,8.29047e+11,2.84406e+11,6.33026e+11,8.23569e+11,6.4962e+11,5.57999e+11,4.11011e+11,6.51762e+11,2.73873e+11,NaN,6.5824e+11,8.6609e+11,5.00265e+11,5.59205e+11,NaN,3.81785e+11,1.14176e+12,5.45498e+11,NaN,5.34163e+11,3.68259e+11,3.88595e+11,2.00154e+11,NaN,3.73208e+11,6.22843e+11,1.5142e+11,4.58232e+11,4.34631e+11,7.02028e+11,3.80677e+11,7.1427e+11,5.28917e+11,2.90706e+11,6.34412e

In [0]:
for i in week_cap.columns:
  try:
    week_cap['{}'.format(i)] = week_cap['{}'.format(i)].astype('float')
  except ValueError:
    pass

In [0]:
week_cap_2 = week_cap.copy()
week_cap_2.drop('14Y_week18',inplace=True)
week_cap_2.loc[week_cap_2.isnull()['삼성전자'],:].index

Index([], dtype='object')

In [0]:
week_cap_2.dropna

<bound method DataFrame.dropna of                    삼성전자          현대차  ...         두산건설 광주은행
14Y_week1   1.02157e+10  4.41012e+13  ...   8.8062e+12  NaN
14Y_week2   1.01219e+10  4.40047e+13  ...  8.78044e+12  NaN
14Y_week3    1.0286e+10  4.45837e+13  ...  8.78044e+12  NaN
14Y_week4   1.00984e+10  4.31361e+13  ...  8.78044e+12  NaN
14Y_week5   9.94209e+09  4.46802e+13  ...   7.3385e+12  NaN
...                 ...          ...  ...          ...  ...
19Y_week48  7.42389e+12  2.67636e+13  ...  8.78887e+10  NaN
19Y_week49  7.54173e+12   2.6323e+13  ...  7.63088e+10  NaN
19Y_week50  8.05727e+12  2.62129e+13  ...  8.04657e+10  NaN
19Y_week51  8.17511e+12  2.69839e+13  ...  8.34349e+10  NaN
19Y_week52   8.2193e+12  2.65433e+13  ...  8.22472e+10  NaN

[311 rows x 220 columns]>

In [0]:
week_cap_2.isnull().sum()

삼성전자        0
현대차         0
SK하이닉스      0
한국전력        0
NAVER       0
         ... 
동국제강        0
대웅          0
농심홀딩스       0
두산건설        0
광주은행      311
Length: 220, dtype: int64

In [0]:
week_cap.loc['14Y_week18']

삼성전자      1.05518e+10
현대차       4.41012e+13
SK하이닉스    2.67882e+13
한국전력      2.31617e+13
NAVER     4.50026e+12
             ...     
동국제강      5.13952e+11
대웅         7.5293e+10
농심홀딩스     4.32267e+11
두산건설      7.46722e+12
광주은행              NaN
Name: 14Y_week18, Length: 220, dtype: object

In [0]:
week_cap['Date'] = week_price['Date']
week_cap['Week'] = week_cap.index

## 1.5. 파일 다운로드

In [0]:
week_cap.to_csv('Mkt_Cap.csv',encoding='cp949',sep=',')
week_price.to_csv('Week_Price.csv',encoding='cp949',sep=',')
week_return.to_csv('Week_Return.csv',encoding='cp949',sep=',')

In [0]:
files.download('Mkt_Cap.csv')
files.download('Week_Price.csv')
files.download('Week_Return.csv')